In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#if you do not have access to the twenty qubit machine, use the publically available backend instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the ibmqx2 instead
device = provider.get_backend('ibmqx2')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 1
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    n_registers: 1
    description: 5 qubit device
    allow_q_object: True
    conditional: False
    max_experiments: 75
    allow_q_circuit: False
    url: None
    allow_object_storage: True
    credits_required: True
    memory: True
    sample_name: sparrow
    backend_name: ibmqx2
    open_pulse: False
    quantum_volume: 8
    online_date: 2017-01-24T05:00:00+00:00
    max_shots: 8192
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 34.92678 µs / 36.50602 µs / 0.0 / 0.00058 / 0.00117 / 0.014
    Q1 / 5.23782 GHz / 57.77644 

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
transQcList = []

In [6]:
def toffoli():
    #Toffoli gate without assertion:
    qr = QuantumRegister(3)
    cr = ClassicalRegister(3)

    circuit = QuantumCircuit(qr, cr)

    circuit.x(qr[1])
    circuit.x(qr[2])
    circuit.ccx(qr[2], qr[1], qr[0])
    circuit.measure(qr, cr)
    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots
def toffoli_assertion():
    #Toffoli gate with assertion:
    qr = QuantumRegister(3)
    cr = ClassicalRegister(3)

    circuit = QuantumCircuit(qr, cr)

    circuit.x(qr[1])
    circuit.x(qr[2])
    circuit.ccx(qr[2], qr[1], qr[0])
    qubitList = [qr[2]]
    success = classical_assertion(circuit, qubitList, value = 1)
    circuit.measure(qr, cr)

    trans_qc = transpile(circuit, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    return trans_qc, num_cnots

In [7]:
def toffoliExperiment():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    num_cnots_original = -1
    for i in range(10):
        trans_qc, num_cnots = toffoli()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = toffoli()
        if num_cnots == min_cnots:
            print("\nnumber of CNOTs in circuit without assertion: ", num_cnots)
            num_cnots_original = num_cnots
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = toffoli_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = toffoli_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 15:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break

In [8]:
toffoliExperiment()


number of CNOTs in circuit without assertion:  6
number of CNOTs in circuit with assertion:  10
    Percentage of the assertion circuit size in total size:  40.0 %


In [9]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of toffoli gate without assertion:

In [10]:
calcSuccessrate(res.get_counts(0), ['111'], 0)

total_count =  8192 success_count =  4780 success_rate =  58.349609375 %


Success rate of toffoli with assertion:

In [11]:
calcSuccessrate(res.get_counts(1), ['0 111'], 1)

total_count =  7336 success_count =  4887 success_rate =  66.61668484187568 %
